In [2]:
import sys
sys.path.insert(0, "..")

import numpy as np
from matplotlib import pyplot as plt

from module.base.network import Network

from module.simulation.meanfield import MeanField
from module.simulation.quick_meanfield2 import QuickMeanField2
from module.simulation.meanfield2 import MeanField2
from module.simulation.set_meanfield2 import SetMeanField2

import module.components.CONST as CONST
from module.components.lawrence_dist import LawrenceDist

from module.components.Adam import Adam

from scipy.integrate import odeint

In [5]:
net = Network(4,4,1,[[0,0,0], [3,3,0]])
net.set_voltage_config([-0.1, 0.1], 0)
mf = QuickMeanField2(net)